In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
import spatialde

sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.10.1 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.13.5 python-igraph==0.10.4 louvain==0.8.0 pynndescent==0.5.8


In [ ]:
# Data
'''
├── filtered_feature_bc_matrix
│   ├── barcodes.tsv.gz
│   ├── features.tsv.gz
│   └── matrix.mtx.gz
├── filtered_feature_bc_matrix.h5  #实际上需要的文件①
├── metrics_summary.csv
├── molecule_info.h5
├── possorted_genome_bam.bam
├── possorted_genome_bam.bam.bai
├── raw_feature_bc_matrix
│   ├── barcodes.tsv.gz
│   ├── features.tsv.gz
│   └── matrix.mtx.gz
├── raw_feature_bc_matrix.h5
├── spatial  # 实际上需要的文件②；记录空间信息
│   ├── aligned_fiducials.jpg 
│   ├── detected_tissue_image.jpg
│   ├── scalefactors_json.json
│   ├── tissue_hires_image.png
│   ├── tissue_lowres_image.png
│   └── tissue_positions_list.csv
└── web_summary.html
'''

In [ ]:
os.chdir('D:/Data/') # Change pathway

# import data
# 载入多文件(Non-spatial scRNA-seq)
# A = sc.read_10x_mtx("GEX1_filtered_feature_bc_matrix", var_names='gene_symbols', cache=True)
# B = sc.read_10x_mtx("GEX2_filtered_feature_bc_matrix", var_names='gene_symbols', cache=True) 
# C = sc.read_10x_mtx("GEX3_filtered_feature_bc_matrix", var_names='gene_symbols', cache=True) 

# 载入多文件(spatial scRNA-seq)
A  = sc.read_visium("GEX1_filtered_feature_bc_matrix")
B  = sc.read_visium("GEX2_filtered_feature_bc_matrix")

A.obs['sample_name'] = "A"
B.obs['sample_name'] = "B"
# C.obs['sample_name'] = "C"

In [ ]:
# merge datasets
# Non-spatial scRNA-seq
# adata = A.concatenate(B)
# adata = A.concatenate(B, C)

# Merge spatial scRNA-seq after QC!
'''
import scanorama
adatas = [A, B]
adatas_cor = scanorama.correct_scanpy(adatas, return_dimred=True)

adatas_cor
len(adatas_cor) # 2
adatas_cor[0].obsm['X_scanorama'].shape

# 使用uns_merge="unique"策略连接两个数据集是为了在连接的anndata对象中保留visium数据集中的两个images
adata_spatial = sc.concat(
    adatas_cor,
    label="library_id",
    uns_merge="unique",
    keys=[
        k
        for d in [
            adatas_cor[0].uns["spatial"],
            adatas_cor[1].uns["spatial"],
        ]
        for k, v in d.items()
    ],
    index_unique="-",
)

sc.pp.neighbors(adata_spatial, use_rep="X_scanorama")
sc.tl.umap(adata_spatial)
sc.tl.leiden(adata_spatial, key_added="clusters")

sc.pl.umap(
    adata_spatial, color=["clusters", "library_id"], palette=sc.pl.palettes.default_20
)

# 在空间坐标中可视化聚类结果
# 首先需要将簇颜色保存在字典中
# 然后在前后两个数据集的视图中可视化，并将结果并排显示
clusters_colors = dict(
    zip([str(i) for i in range(18)], adata_spatial.uns["clusters_colors"])
)

fig, axs = plt.subplots(1, 2, figsize=(15, 10))

for i, library in enumerate(
    ["V1_Mouse_Brain_Sagittal_Anterior", "V1_Mouse_Brain_Sagittal_Posterior"]
):
    ad = adata_spatial[adata_spatial.obs.library_id == library, :].copy()
    sc.pl.spatial(
        ad,
        img_key="hires",
        library_id=library,
        color="clusters",
        size=1.5,
        palette=[
            v
            for k, v in clusters_colors.items()
            if k in ad.obs.clusters.unique().tolist()
        ],
        legend_loc=None,
        show=False,
        ax=axs[i],
    )

plt.tight_layout()

'''

In [ ]:
# 格外标记分组
labels_sample = {
    'A': 'a',
    'B': 'b'
}

# 基于labels_sample和adata的sample_name对应关系进行格外标记
adata.obs['CAR'] = pd.Series([labels_sample[sample_name] for sample_name in adata.obs['sample_name']], index=adata.obs.index)

# 根据sample_name把barcode重命名
barcode = list(map(lambda x: x[0] + '-' + x[1], zip(adata.obs["sample_name"], adata.obs.index)))
# make barcode unique 以避免warning
adata.obs.index = barcode
# adata.var_names_make_unique()

In [ ]:
# 保存初步读入h5ad数据
# os.chdir('D:/Data')
adata.write('Original_Data.h5ad')

In [ ]:
# 将线粒体基因组保存为注释 var.mt
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# 计算指标, qc的var选择 var.mt
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
# n_genes_by_counts：每个细胞中，有表达的基因的个数；
# total_counts：每个细胞的基因总计数（总表达量）；
# pct_counts_mt：每个细胞中，线粒体基因表达量占该细胞所有基因表达量的百分比

In [ ]:
# QC and preprocessing
fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.distplot(adata.obs["total_counts"], kde=False, ax=axs[0])# total_counts 代表一个细胞的基因总表达量
sns.distplot(adata.obs["total_counts"][adata.obs["total_counts"] < 10000], kde=False, bins=40, ax=axs[1]) # 节选部分数据可视化
sns.distplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2]) # n_genes_by_counts 代表一个细胞中，有表达的基因的个数
sns.distplot(adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000], kde=False, bins=60, ax=axs[3]) # 节选部分数据可视化

In [ ]:
# 保留 total_counts 在5000到35000的细胞
sc.pp.filter_cells(adata, min_counts=5000)
sc.pp.filter_cells(adata, max_counts=35000)

# 保留线粒体基因 pct_counts_mt 占比小于20%的细胞
adata = adata[adata.obs["pct_counts_mt"] < 20]
print(f"#cells after MT filter: {adata.n_obs}")

# 保留在大于10个细胞表达的基因
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
# 标准化以及对数化
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

# 检测高变基因
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)

In [ ]:
# pca降维，计算neighbors graph，umap降维，leiden聚类
sc.pp.pca(adata)
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# 基于PCA的结果选择合适的PCA数量
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
# leiden聚类效果不好可以使用sc.tl.louvain
sc.tl.leiden(adata, key_added="clusters")

In [ ]:
# 保存h5ad数据
# os.chdir('D:/Data')
adata.write('Umap_Data.h5ad')

In [ ]:
# 基于umap可视化一些相关变量
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

In [ ]:
# 查看空间坐标
adata.obsm['spatial'].shape
# adata.obsm['spatial']


In [ ]:
# total_counts 和 n_genes_by_counts 在空间坐标中的分布
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts"])

In [ ]:
# clusters在空间坐标中的分布
sc.pl.spatial(adata, img_key="hires", color="clusters", size=1.5)

In [ ]:
# 提高透明度，以结合染色图片与转录组数据
sc.pl.spatial(adata, img_key="hires", color="clusters", groups=["0", "4"], alpha=0.5, size=1.3)

In [ ]:
# 选择感兴趣的簇，计算标记基因并绘制一个热图，图中显示了簇中前10个标记基因的表达水平
sc.tl.rank_genes_groups(adata, "clusters", method="t-test")
sc.pl.rank_genes_groups_heatmap(adata, groups="4", n_genes=10, groupby="clusters")


In [ ]:
# 结合标记基因，在染色图片中分析
sc.pl.spatial(adata, img_key="hires", color=["clusters", "CR2"])

In [ ]:
# SpatialDE 识别空间变异基因
%%time
counts = pd.DataFrame(adata.X.todense(), columns=adata.var_names, index=adata.obs_names)
coord = pd.DataFrame(adata.obsm['spatial'], columns=['x_coord', 'y_coord'], index=adata.obs_names)

In [ ]:
# results中会保存基于空间转录组数据计算得到的可变基因
results = SpatialDE.run(coord, counts) # 需要运行很久
results.head().T

In [ ]:
# 进一步查看差异较大的基因
results.sort_values('qval').head(10)[['g', 'l', 'qval']]

In [ ]:
# 检测到一些空间差异表达的基因，例如 A 和 B
# 可视化这些基因的一种简单方法是绘制上述 x 和 y 坐标，但让颜色对应于表达水平
figsize(10, 3)
for i, g in enumerate(['A', 'B', 'Marker']):
    plt.subplot(1, 3, i + 1)
    plt.scatter(sample_info['x'], sample_info['y'], c=norm_expr[g]);
    plt.title(g)
    plt.axis('equal')
    plt.colorbar(ticks=[]);

In [ ]:
# 查看差异小的基因并绘制图作为对照
results.sort_values('qval').tail(10)[['g', 'l', 'qval']]
figsize(10, 3)
for i, g in enumerate(['D', 'E', 'F']):
    plt.subplot(1, 3, i + 1)
    plt.scatter(sample_info['x'], sample_info['y'], c=norm_expr[g]);
    plt.title(g)
    plt.axis('equal')
    plt.colorbar(ticks=[]);


In [ ]:
# 由于无法通过火山图研究显著性，可以研究由空间变化解释的方差分数
figsize(5, 4)
plt.yscale('log')

plt.scatter(results['FSV'], results['qval'], c='black')

plt.axhline(0.05, c='black', lw=1, ls='--');

plt.gca().invert_yaxis();
plt.xlabel('Fraction spatial variance')
plt.ylabel('Adj. P-value');


In [ ]:
# 自动表达组织学
# To perform automatic expression histology (AEH), the genes should be filtered by SpatialDE significance
# For this example, use a very weak threshold. But in typical use, filter by qval < 0.05
sign_results = results.query('qval < 0.5')
sign_results['l'].value_counts()
histology_results, patterns = SpatialDE.aeh.spatial_patterns(X, resid_expr, sign_results, C=3, l=1.8, verbosity=1)
figsize(10, 3)
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.scatter(sample_info['x'], sample_info['y'], c=patterns[i]);
    plt.axis('equal')
    plt.title('Pattern {} - {} genes'.format(i, histology_results.query('pattern == @i').shape[0] ))
    plt.colorbar(ticks=[]);


In [ ]:
# 空间基因模式
for i in histology_results.sort_values('pattern').pattern.unique():
    print('Pattern {}'.format(i))
    print(', '.join(histology_results.query('pattern == @i').sort_values('membership')['g'].tolist()))
    print()
# 空间高变基因，最后落回基因功能和异质性上，甚至涉及到细胞之间的通讯

In [ ]:
# 基因集打分
# scanpy.tl.score_genes
gene_list = ['A', 'B', 'C']
sc.tl.score_genes(adata, gene_list, ctrl_size=50, gene_pool=None, n_bins=25, score_name='score', random_state=0, copy=False, use_raw=None)

# tl.score_genes_cell_cycle用给定S phase 和 G2M phase的两个基因集，计算打分，然后根据得分分配细胞phase


In [ ]:
# 细胞周期打分，在同一簇细胞中，不同位置的细胞差异可能是细胞周期差异，选择性做该分析
s_genes_file = data_dir + 's_genes_tirosh_hm.txt'
g2m_genes_file = data_dir + 'g2m_genes_tirosh_hm.txt'

s_genes = pd.read_table(s_genes_file, header = None).values.flatten()
g2m_genes = pd.read_table(g2m_genes_file, header = None).values.flatten()

s_genes_hvg = adata.var_names[np.in1d(adata.var_names, s_genes)]
g2m_genes_hvg = adata.var_names[np.in1d(adata.var_names, g2m_genes)]

adata.obs['S_score']= np.zeros(adata.shape[0])
adata.obs['G2M_score'] = np.zeros(adata.shape[0])
adata.obs['phase'] = np.zeros(adata.shape[0])

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_hvg, g2m_genes=g2m_genes_hvg)

print(len(s_genes_hvg))
print(len(g2m_genes_hvg))

In [ ]:
adata.obs['phase'].value_counts()

In [ ]:
# Before we save the data to file, we convert the gene expression matrix X to the sparse format to memory.
import scipy.sparse as sparse

adata.X = sparse.csr_matrix(adata.X)
adata.write('data_processed.h5ad')

In [ ]:
# 细胞亚型注释
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

sc.pl.dotplot(adata=, 
              var_names =,
              groupby=, 
              use_raw=False)

sc.pl.heatmap(adata=, var_names=, 
              figsize=(5,10),
              groupby=, 
              use_raw=False, vmin=0)

sc.pl.matrixplot(adata=, var_names=,
                 groupby=, 
                 use_raw=False, vmin=0)

sc.pl.stacked_violin(adata = ,var_names = , groupby=, 
                     use_raw=False)

In [ ]:
adata.obs['annotated'] = adata.obs['louvain_r1.5'].cat.add_categories(['CD4 T cells', 
                        'CD14+ Monocytes', 'B cells', 'CD8 T cells', 
                        'FCGR3A+ Monocytes', 'NK cells', 'Dendritic cells', 'Megakaryocytes'])

adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'CD4 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'CD14+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'B cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'CD8 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'FCGR3A+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'NK cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'Dendritic cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['cluster name'])] = 'Megakaryocytes'

#remove unused categories from annotation
adata.obs['annotated'] = adata.obs['annotated'].cat.remove_unused_categories()

adata.obs['annotated'].value_counts()

In [ ]:
sc.pl.umap(adata, color='annotated', legend_loc='on data', title='', frameon=False)
sc.pl.umap(adata, color='annotated',  title='', frameon=True)

sc.pl.dotplot(adata=, 
              var_names =,
              groupby=, 
              use_raw=False)

sc.pl.heatmap(adata=, var_names=, 
              figsize=(5,10),
              groupby=, 
              use_raw=False, vmin=0)

sc.pl.matrixplot(adata=, var_names=,
                 groupby=, 
                 use_raw=False, vmin=0)

sc.pl.stacked_violin(adata = ,var_names = , groupby=, 
                     use_raw=False)

In [ ]:
adata.write('data_annoated.h5ad')

In [ ]:
# PAGA
sc.tl.paga(adata = adata, groups='annotated')

rcParams['figure.figsize']=(7,7)
sc.pl.paga_compare(adata = adata, basis='umap', frameon=True)

In [ ]:
adata.write('data_PAGA.h5ad')

In [ ]:
# Pseudotime 拟时序分析
# 选择细胞亚群
adata_mono = adata[np.in1d(adata.obs['annotated'], 
                           ['CD14+ Monocytes', 'FCGR3A+ Monocytes'])].copy()

sc.tl.pca(adata_mono, svd_solver='arpack')
sc.pp.neighbors(adata_mono)

# Convert UMAP indices to arrays.
umap_0 = [term[0] for term in adata_mono.obsm['X_umap']]
umap_1 = [term[1] for term in adata_mono.obsm['X_umap']]

In [ ]:
# Set root cell to the cell with the smallest value in the first UMAP component and compute DPT.
adata_mono.uns['iroot'] = np.flatnonzero(umap_0== max(umap_0))[0]
sc.tl.dpt(adata = adata_mono)

In [ ]:
# Visualise DPT on a UMAP and on a diffusion map.
rcParams['figure.figsize']=(7,7)
sc.pl.umap(adata_mono, color=['dpt_pseudotime', 'annotated'])

rcParams['figure.figsize']=(7,7)
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'annotated'], components=['1,2'])
sc.pl.diffmap(adata_mono, color=['dpt_pseudotime', 'annotated'], components=['1,3'])

In [ ]:
# Run a differential test on the two groups of monocytes in order to determine characteristic genes.  
sc.tl.rank_genes_groups(adata_mono, groupby='annotated', 
                        groups= ['FCGR3A+ Monocytes'], reference='CD14+ Monocytes', rankby_abs=True)

rcParams['figure.figsize']=(10,5)
sc.pl.rank_genes_groups(adata_mono, size=10, n_genes=30)

In [ ]:
rcParams['figure.figsize']= (15,5)
sc.pl.rank_genes_groups_violin(adata_mono, use_raw=False)

In [ ]:
mono_genes = [idx[1][0] for idx in enumerate(adata_mono.uns['rank_genes_groups']['names'])]

# In order to visualise the gene expression along pseudotime, we have to compute PAGA for the two groups of monocytes.
sc.tl.paga(adata_mono, groups='annotated')

In [ ]:
# Modify the format of the data matrix, because `paga_path` takes only dense matrices
adata_mono.X = adata_mono.X.todense()

rcParams['figure.figsize']=(20,10)
sc.pl.paga_path(adata_mono, nodes=['FCGR3A+ Monocytes','CD14+ Monocytes'], 
                keys=mono_genes[:25],n_avg=10, use_raw=False, save='_monocyte_transition.pdf')